In [168]:
import pandas as pd
import numpy as np

In [169]:
df_cpi = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\cpi_index_VN.csv')
df_fdi = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\fdi_inflow_VN.csv')
df_gdp = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\gdp_growth_VN.csv')
df_retail = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\Retail_processed.csv.csv')
df_unemployment = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\Unemployment_processed.csv')

In [170]:
def prepare_data(df, date_col='date', value_col=None, start_year=2010, end_year=2024):
    """Đảm bảo cột ngày tháng ở định dạng datetime và thiết lập làm index."""
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col)
    if value_col is None:
        value_col = df.columns[0]
    df = df[[value_col]].rename(columns={value_col: 'annual_value'})
    #  Sort index theo thứ tự thời gian
    df = df.sort_index()
    #Lọc năm
    df = df[(df.index.year >= start_year) & (df.index.year <= end_year)]
    return df

# Chuẩn bị dữ liệu
data_frames = {
    'fdi_inflow': prepare_data(df_fdi, value_col='fdi_inflow'),
    'cpi_index': prepare_data(df_cpi, value_col='cpi_index'),
    'gdp_growth': prepare_data(df_gdp, value_col='gdp_percent'),
    'retail_value': prepare_data(df_retail, value_col='retail_value'),
    'unemployment_value': prepare_data(df_unemployment, value_col='unemployment_value')
}


# Định nghĩa tham số dựa trên bản chất của biến số
DISAGG_PARAMS = {
    # Dòng chảy (Flow): Tổng 4 quý = giá trị năm
    'fdi_inflow': {'agg_func': 'sum'}, 
    # Tỷ lệ/Chỉ số (Rate/Index): Trung bình 4 quý = giá trị năm
    'cpi_index': {'agg_func': 'mean'},
    'gdp_growth': {'agg_func': 'mean'},
    'retail_value': {'agg_func': 'mean'},
    'unemployment_value': {'agg_func': 'mean'}
}

In [171]:
def benchmarked_linear_disaggregation(df_annual, agg_func):
    """
    Phân tách dữ liệu năm thành dữ liệu quý sử dụng phương pháp 
    nội suy tuyến tính được chuẩn hóa (benchmarking).
    """
    value_col = 'annual_value'
    
    # Đảm bảo index được sort
    df_annual = df_annual.sort_index()
    

    # 1. UPSAMPLING VÀ NỘI SUY CƠ SỞ
    # Chuyển về đầu năm (YYYY-01-01) để resample chính xác
    df_annual_start = df_annual.copy()
    df_annual_start.index = df_annual_start.index.to_period('Y').to_timestamp(how='start')
    
  
    # Resample thành quý bắt đầu và điền forward, sau đó nội suy
    df_quarterly_resampled = df_annual_start.resample('QS').asfreq()

    # Nội suy tuyến tính
    df_quarterly_interpolated = df_quarterly_resampled.interpolate(method='linear')
    df_quarterly_interpolated.columns = ['interpolated_value']
    
    # 2. TÍNH TOÁN CHUẨN HÓA (Benchmarking)
    
    # Tạo key năm để map dữ liệu
    df_annual_temp = df_annual.copy()
    df_annual_temp['annual_period'] = df_annual_temp.index.to_period('Y')
    df_quarterly_interpolated['annual_period'] = df_quarterly_interpolated.index.to_period('Y')
    
    # Map giá trị năm gốc
    annual_map = df_annual_temp.set_index('annual_period')[value_col]
    
    # Tính tổng/trung bình nội suy theo năm
    agg_quarterly = df_quarterly_interpolated.groupby('annual_period')['interpolated_value'].agg(agg_func)
    
    # Tạo dataframe điều chỉnh
    df_annual_adjustment = pd.DataFrame({
        'annual_value': annual_map,
        'agg_interpolated': agg_quarterly
    }).dropna()
    

    # 3. TÍNH HỆ SỐ ĐIỀU CHỈNH
    if agg_func == 'sum':
        # Với Sum: adjustment là hệ số nhân (ratio)
        df_annual_adjustment['adjustment'] = (
            df_annual_adjustment['annual_value'] / 
            df_annual_adjustment['agg_interpolated'].replace(0, np.nan)
        )
    else:  # mean
        # Với Mean: adjustment là chênh lệch (difference)
        df_annual_adjustment['adjustment'] = (
            df_annual_adjustment['annual_value'] - 
            df_annual_adjustment['agg_interpolated']
        )
    
    # 4. HỢP NHẤT VÀ ĐIỀU CHỈNH
    df_quarterly_result = df_quarterly_interpolated.reset_index().copy()
    df_quarterly_result['annual_period'] = df_quarterly_result['annual_period'].astype(str)
    
    adj_reset = df_annual_adjustment['adjustment'].reset_index()
    adj_reset['annual_period'] = adj_reset['annual_period'].astype(str)
    
    df_quarterly_result = pd.merge(
        df_quarterly_result,
        adj_reset[['annual_period', 'adjustment']],
        on='annual_period',
        how='left'
    )
    
    # Áp dụng điều chỉnh
    if agg_func == 'sum':
        df_quarterly_result['quarterly_value'] = (
            df_quarterly_result['interpolated_value'] * 
            df_quarterly_result['adjustment']
        )
    else:  # mean
        df_quarterly_result['quarterly_value'] = (
            df_quarterly_result['interpolated_value'] + 
            df_quarterly_result['adjustment']
        )
    
    # 5. LỌC KẾT QUẢ VÀ CĂN CHỈNH INDEX
    result_series = df_quarterly_result.set_index('date')['quarterly_value'].dropna()
    
    # Lấy phạm vi năm từ dữ liệu gốc
    first_year = df_annual.index.min().year
    last_year = df_annual.index.max().year

    # Lọc chỉ giữ dữ liệu trong phạm vi năm hợp lệ
    result_series = result_series[
        (result_series.index.year >= first_year) & 
        (result_series.index.year <= last_year)
    ]
    
    print(f"  Số quý sau lọc: {len(result_series)}")
    
    return result_series


In [172]:

# --- ÁP DỤNG VÀ IN KẾT QUẢ ---

results = {}
for name, params in DISAGG_PARAMS.items():
    print(f"Đang xử lý: {name}")
    results[f'{name}_quarterly'] = benchmarked_linear_disaggregation(
        data_frames[name].copy(), 
        agg_func=params['agg_func']
    )

# Kết hợp tất cả dữ liệu
output_df = pd.concat(results.values(), axis=1, join='outer')
output_df.columns = results.keys()

# Lưu kết quả
output_df.to_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\disaggregated_quarterly_data_fixed.csv')


Đang xử lý: fdi_inflow
  Số quý sau lọc: 57
Đang xử lý: cpi_index
  Số quý sau lọc: 57
Đang xử lý: gdp_growth
  Số quý sau lọc: 57
Đang xử lý: retail_value
  Số quý sau lọc: 49
Đang xử lý: unemployment_value
  Số quý sau lọc: 57


In [173]:

# 1. Tải các tệp dữ liệu hàng ngày (Daily/Sparse)
df_oil = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\OIL_processed.csv.csv')
df_usdvnd = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\USDVND_processed.csv')
df_vnindex = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\VNINDEX_processed.csv')

# --- 2. Hàm Chuẩn bị và Làm sạch Dữ liệu Hàng ngày ---
def prepare_daily_data(df, date_col='date'):
    """
    Chuyển đổi cột ngày tháng, thiết lập Index, và xử lý trùng lặp ngày tháng
    (Giữ lại quan sát cuối cùng cho mỗi ngày).
    """
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col)
    
    # Xử lý các ngày trùng lặp (nếu có, giữ lại giá trị cuối cùng)
    df = df.loc[~df.index.duplicated(keep='last')]
    
    # Sắp xếp Index theo thời gian
    df = df.sort_index()
    return df

# Áp dụng hàm làm sạch
df_oil_clean = prepare_daily_data(df_oil)
df_usdvnd_clean = prepare_daily_data(df_usdvnd)
df_vnindex_clean = prepare_daily_data(df_vnindex)

# 3. Hợp nhất Dữ liệu Hàng ngày (Outer Join)
# Outer Join đảm bảo giữ lại tất cả các ngày từ VNINDEX và các điểm thưa thớt từ OIL/USDVND
df_daily_merged = pd.merge(
    df_oil_clean,
    df_usdvnd_clean,
    left_index=True,
    right_index=True,
    how='outer'
)
df_daily_merged = pd.merge(
    df_daily_merged,
    df_vnindex_clean,
    left_index=True,
    right_index=True,
    how='outer'
)

# 4. Tổng hợp sang Tần suất Quý ('Q')
# Định nghĩa Quy tắc Tổng hợp (Aggregation Rules) dựa trên phân tích kinh tế
agg_rules = {
    # Mean: Cho giá hàng hóa, giúp làm phẳng biến động
    'oil_price_close': 'mean',      
    # Last: Cho Tỷ giá/Chỉ số, đại diện cho giá trị cuối kỳ
    'usd_vnd_close': 'last',        
    'vnindex_close': 'last'         
}

# Tổng hợp (Resample) sang tần suất Quý (kết thúc Quý) và áp dụng quy tắc
df_quarterly = df_daily_merged.resample('QS').agg(agg_rules)


df_quarterly_filtered = df_quarterly[
    (df_quarterly.index.year >= 2010) & 
    (df_quarterly.index.year <= 2024)
]


# 5. Lưu và In Kết quả
df_quarterly_filtered.to_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\merged_daily_to_quarterly_final.csv')


In [174]:
df_daily_to_quarterly = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\merged_daily_to_quarterly_final.csv' )
df_annual_to_quarterly = pd.read_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\processed\disaggregated_quarterly_data_fixed.csv' )

In [175]:
df_final = pd.merge(
    df_daily_to_quarterly,
    df_annual_to_quarterly,
    left_on='date',
    right_on='date',
    how='inner'  # Chỉ giữ lại các quý có trong cả hai bộ dữ liệu
)

In [176]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   date                          57 non-null     object 
 1   oil_price_close               37 non-null     float64
 2   usd_vnd_close                 37 non-null     float64
 3   vnindex_close                 57 non-null     float64
 4   fdi_inflow_quarterly          57 non-null     float64
 5   cpi_index_quarterly           57 non-null     float64
 6   gdp_growth_quarterly          57 non-null     float64
 7   retail_value_quarterly        49 non-null     float64
 8   unemployment_value_quarterly  57 non-null     float64
dtypes: float64(8), object(1)
memory usage: 4.1+ KB


In [177]:
df_final['date'] = pd.to_datetime(df_final['date'])
df_final.set_index('date', inplace=True)

In [179]:
df_final = df_final[
    (df_final.index.year >= 2015) & 
    (df_final.index.year <= 2024)
]


In [180]:
df_final

,oil_price_close,usd_vnd_close,vnindex_close,fdi_inflow_quarterly,cpi_index_quarterly,gdp_growth_quarterly,retail_value_quarterly,unemployment_value_quarterly
date,,,,,,,,
2015-01-01,55.025246,21188.0,551.13,2.876860e+09,-0.132692,7.098584,110.70000,1.848000
2015-04-01,63.393333,21780.0,593.05,2.925620e+09,0.376570,7.024297,110.10000,1.848000
2015-07-01,51.177969,22112.0,562.64,2.974380e+09,0.885832,6.950011,109.50000,1.848000
2015-10-01,44.638438,22185.0,579.03,3.023140e+09,1.395094,6.875725,108.90000,1.848000
2016-01-01,35.243571,21950.0,561.22,3.015385e+09,2.348745,6.596191,105.93750,1.838250
2016-04-01,47.062969,21911.0,632.26,3.105128e+09,2.561747,6.658736,106.91250,1.844750
2016-07-01,46.950781,21797.0,685.73,3.194872e+09,2.774749,6.721282,107.88750,1.851250
2016-10-01,51.072951,22468.0,664.87,3.284615e+09,2.987751,6.783827,108.86250,1.857750
2017-01-01,54.496452,22435.0,722.31,3.398462e+09,3.512993,6.743384,111.75375,2.141375


In [182]:
df_final.to_csv(r'C:\Users\Admin\Desktop\TANPHAT\hocotruong\Năm ba 2025-2026\HK1_A\Thu thập và tiền xử lý dữ liệu\Đồ_án_GDP\data\final_clean_dataset.csv')